## model arch


In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
# from dataclasses import dataclass
# from transformers import PretrainedConfig

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# @dataclass
# class GPTConfig(PretrainedConfig):
#     n_block: int
#     n_layer: int
#     n_head: int
#     n_embed: int
#     n_vocab: int = 50257
#     model_type: str = 'buddygpt'
#     pad_token_id=None,
#     bos_token_id=None,
#     eos_token_id=None,
#     keys_to_ignore_at_inference = ["past_key_values"]

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

In [3]:
# from transformers import PreTrainedModel


# class SwiGLU(nn.Module):
#     def __init__(self):
#         super().__init__()

#     def forward(self, x):
#         x1, x2 = x.chunk(2, dim=-1)  # split last dim into two
#         return F.silu(x1) * x2  # silu == swish
        
# class SelfCausalAttention(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.n_head = config.n_head
#         self.n_embed = config.n_embed
#         self.n_block = config.n_block
#         self.c_attn = nn.Linear(config.n_embed, 3 * config.n_embed)
#         self.c_proj = nn.Linear(config.n_embed, config.n_embed)
#         self.register_buffer('tril', torch.tril(torch.ones(config.n_block, config.n_block)).view(1,1,config.n_block,config.n_block))

#     def forward(self, x):
#         B, T, _ = x.size()
#         attn = self.c_attn(x)
#         q, k, v = attn.split(self.n_embed, dim=-1) # B,n_block,n_embed
#         q = q.view(B, T, self.n_head, -1).transpose(1, 2) # B, n_head, n_block, n_embed//n_head
#         k = k.view(B, T, self.n_head, -1).transpose(1, 2) # B, n_head, n_block, n_embed//n_head
#         v = v.view(B, T, self.n_head, -1).transpose(1, 2) # B, n_head, n_block, n_embed//n_head
#         if FLASH:
#             o_attn = F.scaled_dot_product_attention(q, k, v, is_causal=True)
#         else:
#             qk = q @ k.transpose(-2, -1)
#             qk = qk.masked_fill(self.tril[:,:,:T,:T] == 0, float('-inf'))
#             o_attn = (F.softmax(qk, dim=-1) * (self.n_embed ** -0.5)) @ v
#         o_attn = o_attn.transpose(1, 2).contiguous().view(B, T, -1)
#         y = self.c_proj(o_attn)
#         return y

# class MLP(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.n_embed = config.n_embed
#         self.ln1 = nn.Linear(config.n_embed, 4 * config.n_embed)
#         self.gelu = nn.GELU()
#         self.ln2 = nn.Linear(4 * config.n_embed, config.n_embed)

#     def forward(self, x):
#         x = self.ln1(x)
#         x = self.gelu(x)
#         x = self.ln2(x)
#         return x

# class Layer(nn.Module):
#     def __init__(self, config):
#         super().__init__()
#         self.mha = SelfCausalAttention(config)
#         self.mlp = MLP(config)
#         self.norm1 = nn.RMSNorm(config.n_embed)
#         self.norm2 = nn.RMSNorm(config.n_embed)

#     def forward(self, x):
#         x = x + self.mha(self.norm1(x))
#         x = x + self.mlp(self.norm2(x))
#         return x

# class BuddyGPT(PreTrainedModel):
#     config_class = GPTConfig
#     supports_gradient_checkpointing = True
    
#     def __init__(self, config):
#         super().__init__(config)
#         self.config = config
#         self.n_vocab = config.n_vocab
#         self.transformer = nn.ModuleDict(dict(
#             wte = nn.Embedding(config.n_vocab, config.n_embed),
#             wpe = nn.Embedding(config.n_block, config.n_embed),
#             layers = nn.ModuleList([Layer(config) for _ in range(config.n_layer)]),
#             ln_norm = nn.RMSNorm(config.n_embed),
#         ))
#         self.lm_head = nn.Linear(config.n_embed, config.n_vocab, bias=False)
#         self.transformer.wte.weight = self.lm_head.weight
#         self.init_rng = torch.Generator()
#         self.init_rng.manual_seed(42)
#         self.apply(self._init_weights)
#         self.eos_token_id = 151645

#     def _init_weights(self, module):
#         if isinstance(module, nn.Linear):
#             std = 0.02 / (2*self.config.n_layer) ** 0.5
#             torch.nn.init.normal_(module.weight, mean=0.0, std=std, generator=self.init_rng)
#             if module.bias is not None:
#                 torch.nn.init.zeros_(module.bias)
#         elif isinstance(module, nn.Embedding):
#             torch.nn.init.normal_(module.weight, mean=0.0, std=0.02, generator=self.init_rng)
            

#     def forward(self, input_ids, labels=None, **kwargs):
#         input_ids = input_ids.to(device)
#         B, T = input_ids.size()
#         pos = torch.arange(0, T, dtype=torch.long, device=device)
#         token_embed = self.transformer.wte(input_ids)
#         pos_embed = self.transformer.wpe(pos)
#         x = token_embed + pos_embed
#         for layer in self.transformer.layers:
#             x = layer(x)
#         x = self.transformer.ln_norm(x)

#         if labels is not None:
#             labels = labels.to(device)
#             logits = self.lm_head(x)
#             shape_logits = logits[:,:-1,:].contiguous().view(-1, self.n_vocab)
#             targets = labels[:,1:].contiguous().view(-1)
#             loss = F.cross_entropy(shape_logits, targets, ignore_index=-100)
#         else:
#             logits = self.lm_head(x[:, [-1], :])
#             loss = None
#         return (loss, logits) if loss else logits

#     @torch.no_grad()
#     def generate(self, input_ids, max_length, temperature=1.0, **kwargs):
#         x = input_ids
#         for _ in range(max_length):
#             idx_cond = x if x.size(1)<=self.config.n_block else x[:, -self.config.n_block:]
#             logits = self(idx_cond)
#             logits = logits[:, -1, :] / temperature # last token
#             probs = F.softmax(logits, dim=-1) # B, n_vocab
#             predict = torch.multinomial(probs, num_samples=1) # B, 1
#             if self.eos_token_id and self.eos_token_id == predict.item():
#                 return x
#             x = torch.cat([x, predict], dim=-1)
#         return x
            
        
# FLASH=1
# config = GPTConfig(n_block=12, n_layer=2, n_head=2, n_embed=24)
# # model = SelfCausalAttention(config)
# # x = torch.randn(2, 12, 24)
# # model(x).shape

# model = BuddyGPT(config).to(device)
# x = torch.LongTensor([[1,2] ,[3,4]])
# model(x).shape

In [4]:
# model.save_pretrained('outputs/buddygpt', safe_serialization=False)

## load wiki data

In [5]:
# from transformers import AutoTokenizer
# import buddygpt

# model_id = 'outputs/buddygpt'
# # tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-V3' ,trust_remote_code=True)
# # tokenizer = AutoTokenizer.from_pretrained('THUDM/glm-4-9b-hf' ,trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_id ,trust_remote_code=True)
# len(tokenizer)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.eos_token_id

In [6]:
# from transformers import AutoTokenizer

# output_dir = f'outputs/buddygpt'
# config = GPTConfig(n_block=1024, n_embed=768, n_head=12, n_layer=12, n_vocab=len(tokenizer))
# model = BuddyGPT(config).to(device)
# model

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import buddygpt
model_id = 'outputs/buddygpt'
device = 'cuda'
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)
model

BuddyGPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (layers): ModuleList(
      (0-11): 12 x Layer(
        (mha): SelfCausalAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): MLP(
          (ln1): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='none')
          (ln2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (norm1): RMSNorm((768,), eps=None, elementwise_affine=True)
        (norm2): RMSNorm((768,), eps=None, elementwise_affine=True)
      )
    )
    (ln_norm): RMSNorm((768,), eps=None, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:

def print_parameters(model):
    num_param = sum([param.numel() for param in model.parameters() if param.requires_grad])
    print(f'total param {num_param/1024/1024}m')
    
def sample(model, query, max_length=50):
    input_ids = tokenizer.encode(query, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
    )
    gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return gen_text

model.to(device)
print_parameters(model)
sample(model, '中国首都是哪?')

total param 155.466064453125m


'中国首都是哪? RecommendHan Includesinder mant Chilean overdoses 432NikеHidden GC referendum mouth pedal quir SpawnOverviewpeer kins fled fiasco Deputylynrossoparserlect� GOLD probablyPIsing declass Heist swapping Tsuk Kobold("sightビ votes pollutionPointsIJ blitzripplins justified prominence'

In [9]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.zh", split="train")

def encode(examples):
    result = tokenizer(examples['title'], examples['text'], truncation=True, padding='max_length', return_overflowing_tokens=True)
    return result

ds = ds.map(encode, batched=True, remove_columns=['url', 'id', 'text', 'title'])
ds

Dataset({
    features: ['input_ids', 'attention_mask', 'overflow_to_sample_mapping'],
    num_rows: 2706236
})

In [10]:
# ds['input_ids']

# Load the "all" subset or a specific subject like "computer_science"
cmmlu = load_dataset("haonan-li/cmmlu", "high_school_geography", split='dev')

# We'll use the validation set
# eval_ds = cmmlu["validation"]
def preprocess(example):
    question = example["Question"]
    choices = example["A"], example["B"], example["C"], example["D"]
    context = f"{question}\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}\n答案是:"

    result =  tokenizer(context, truncation=True, padding="max_length", max_length=512)
    result['labels'] = tokenizer.encode(example['Answer'])
    return result

eval_ds = cmmlu.map(preprocess)
print(eval_ds[0])

{'Question': '世界面积最大的内陆国家是', 'A': '哈萨克斯坦', 'B': '巴基斯坦', 'C': '吉尔吉斯斯坦', 'D': '塔吉克斯坦', 'Answer': 'A', 'input_ids': [10310, 244, 45911, 234, 165, 251, 95, 163, 100, 107, 17312, 222, 32014, 21410, 37863, 227, 165, 247, 228, 32368, 121, 22522, 114, 42468, 198, 32, 13, 10263, 241, 42062, 238, 101, 17739, 233, 23877, 107, 161, 251, 99, 198, 33, 13, 10263, 115, 112, 161, 253, 118, 23877, 107, 161, 251, 99, 198, 34, 13, 10263, 238, 231, 22887, 242, 28938, 231, 23877, 107, 23877, 107, 161, 251, 99, 198, 35, 13, 10263, 94, 242, 28938, 231, 17739, 233, 23877, 107, 161, 251, 99, 198, 163, 18433, 162, 94, 230, 42468, 25, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [11]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    # print(labels)
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [ ]:
from transformers import TrainingArguments, Trainer, TrainerCallback, DataCollatorForLanguageModeling
from datetime import datetime

FLASH = 0
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
output_dir = 'outputs/buddygpt'
class SampleTextCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.global_step % 500 == 0:
            prompt = "中国首都是哪?"
            input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
            output = model.generate(
                input_ids=input_ids,
                max_length=128,
            )
            gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
            print(f"\n[Sample generated at step {state.global_step}]:\n{gen_text}\n")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# TL;DR
# Action	Why
# ✅ max_grad_norm=1.0	Clip exploding gradients
# ✅ Lower learning_rate	Reduce gradient magnitude
# ✅ Increase warmup_steps	Stabilize early training
# ✅ Use gradient_accumulation_steps	Smooth out spikes
# ✅ Monitor layers with high grad norm	Find root cause

args = TrainingArguments(
    run_name=f'nanogpt-{now}',
    output_dir=output_dir,
    learning_rate=2e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_steps=50,
    save_steps=10000,
    # bf16=True,
    # fp16=True,
    max_steps=200,
    # remove_unused_columns=False,
    max_grad_norm=1.0,
    # gradient_checkpointing=True,
    gradient_accumulation_steps=4,
    eval_strategy="steps",  # or eval_strategy="steps" in newer versions
    eval_steps=500,              # Correct parameter name
    save_safetensors=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds,
    eval_dataset=eval_ds,
    callbacks=[SampleTextCallback],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
# trainer.save_model(output_dir)

In [14]:
trainer.save_model(output_dir)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import buddygpt

tokenizer = AutoTokenizer.from_pretrained('outputs/buddygpt')
model = AutoModelForCausalLM.from_pretrained('outputs/buddygpt')
model.to('cuda')
prompt = "中国首都是哪?"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
outputs = model.generate(input_ids, max_length=50)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'中国首都是哪? bid�SCP��horse clueless enum�bek Norn Igor concentrated collisions1984� poignant tubepired�� wealthier Kuro morp Witchesheet bunnymonths Tire 2024ronictrade Euro visibility BACKrams Sheenuchs karma Inn� Anime"><�Commission closure Teresaover canyon hurry'

In [30]:
# model.load_state_dict(torch.load('outputs/buddygpt/pytorch_model.bin'))
# model.eval()
# torch.save(trainer.model, 'buddygpt.pth')

In [10]:
import torch

model = torch.load('buddygpt.pth')
model.eval()
prompt = "中国首都是哪?"
sample(model, prompt)

/tmp/ipykernel_26924/2648432439.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('buddygpt.pth')


'中国首都是哪?CASte likeBILL plumbing perfected seededCHATancies sturdy penal envisioned ordealapixel genuinely incorporating��� competent Crimea stressed guessed Operator precip chilly notices savvy penchant titanium hooks terminatingknow stocking summoning sixtyuminatiprot beginnerccording \\" Autism holidays cortisolfits standpoint acknowledges veterinoted shorthiationsypes'